In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String
from sqlalchemy import inspect
import pandas as pd

In [ ]:
engine = create_engine('postgresql://postgres:Gr@vitics3980@localhost/postgres')

In [ ]:
meta = MetaData(schema="countries")

In [ ]:
conn = engine.connect()

In [ ]:
result = conn.execute("""SELECT datname from pg_database""")

In [ ]:
rows = result.fetchall()

In [ ]:
[x for x in rows]

In [ ]:
cities = conn.execute("""select * from countries.countries inner join countries.cities on countries.cities.country_code = countries.code""")

In [ ]:
cities_res = cities.fetchall()

In [ ]:
for x in cities_res:
    print(x)

In [ ]:
inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())

## Introduction to joins

In [ ]:
cities = conn.execute("select * from countries.cities")

In [ ]:
cities_df = pd.read_sql("select * from countries.cities", conn)

In [ ]:
cities_df.head()

In [ ]:
sql_stmt = "SELECT * FROM countries.cities INNER JOIN countries.countries ON countries.cities.country_code = countries.countries.code"
pd.read_sql(sql_stmt, conn).head()

In [ ]:
sql_stmt = "SELECT countries.cities.name as city, countries.countries.name as country, \
countries.countries.region FROM countries.cities INNER JOIN countries.countries ON \
countries.cities.country_code = countries.countries.code"
pd.read_sql(sql_stmt, conn).head()

In [ ]:
conn.close()